In [24]:
from dotenv import load_dotenv
import os 

In [25]:
!pip install great_expectations==0.18.12

In [26]:
# Import necessary libraries
import pandas as pd               # For data manipulation using DataFrames
import numpy as np                # For numerical operations
import matplotlib.pyplot as plt   # For data visualization
import os                         # For operating system-related tasks
import joblib                     # For saving and loading models
import hopsworks                  # For getting access to hopsworks



# Import specific modules from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For data preprocessing
from sklearn.metrics import accuracy_score                        # For evaluating model accuracy

In [27]:
#from alpha_vantage.timeseries import TimeSeries
#import pandas as pd

#load_dotenv()

#api_key = os.environ.get('stocks_api') # Replace this with your actual API key
#ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
#data, meta_data = ts.get_daily(symbol='TSLA', outputsize='full')

#print(data.head())

In [28]:
#data.info()

In [29]:
#meta_data

In [30]:
# Define your file path and name
#file_path = 'TSLA_stock_price.csv'  # Customize the path and filename

# Save the DataFrame to CSV
#stock_data.to_csv(file_path)

#print(f"Data saved to {file_path}")


In [31]:
# Load and display the data from CSV to confirm
tsla_df = pd.read_csv('TSLA_stock_price.csv')
print(tsla_df.head())
    

         date  1. open  2. high    3. low  4. close    5. volume ticker
0  2024-05-03   182.10   184.78  178.4200    181.19   75491539.0   TSLA
1  2024-05-02   182.86   184.60  176.0200    180.01   89148041.0   TSLA
2  2024-05-01   182.00   185.86  179.0100    179.99   92829719.0   TSLA
3  2024-04-30   186.98   190.95  182.8401    183.28  127031787.0   TSLA
4  2024-04-29   188.42   198.87  184.5400    194.05  243869678.0   TSLA


In [32]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549016
Connected. Call `.close()` to terminate connection gracefully.


In [33]:
import re 

In [34]:
def clean_column_name(name):
    # Remove all non-letter characters
    cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    return cleaned_name


In [35]:
tsla_df

,date,1. open,2. high,3. low,4. close,5. volume,ticker
0,2024-05-03,182.10,184.7800,178.4200,181.19,75491539.0,TSLA
1,2024-05-02,182.86,184.6000,176.0200,180.01,89148041.0,TSLA
2,2024-05-01,182.00,185.8600,179.0100,179.99,92829719.0,TSLA
3,2024-04-30,186.98,190.9500,182.8401,183.28,127031787.0,TSLA
4,2024-04-29,188.42,198.8700,184.5400,194.05,243869678.0,TSLA
...,...,...,...,...,...,...,...
3481,2010-07-06,20.00,20.0000,15.8300,16.11,6866900.0,TSLA
3482,2010-07-02,23.00,23.1000,18.7100,19.20,5139800.0,TSLA
3483,2010-07-01,25.00,25.9200,20.2700,21.96,8218800.0,TSLA
3484,2010-06-30,25.79,30.4192,23.3000,23.83,17187100.0,TSLA


In [36]:
# Assuming 'tsla_df' is your DataFrame
tsla_df.columns = [clean_column_name(col) for col in tsla_df.columns]


In [37]:
print(tsla_df.columns)


Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')


In [38]:
import pandas as pd

# Assuming tsla_df is your pandas DataFrame
# Convert the "date" column to timestamp
tsla_df['date'] = pd.to_datetime(tsla_df['date'])


In [39]:
# Define a feature group
tesla_fg = fs.get_or_create_feature_group(
    name="tesla_stock",
    description="Tesla stock dataset from alpha vantage",
    version=1,
    primary_key=["ticker"],
    event_time=['date'],
    online_enabled=False,
)

2024-05-06 11:35:29,224 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [40]:
tesla_fg.insert(tsla_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549016/fs/544838/fg/787790


Uploading Dataframe: 0.00% |          | Rows 0/3486 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: tesla_stock_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/tesla_stock_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x304ac0c90>, None)

In [41]:
news_df = pd.read_csv('news_articles_ema.csv')


In [42]:
news_df_updated = news_df.drop(columns=['exp_mean_7_days'])

In [43]:
news_df_updated['date'] = pd.to_datetime(news_df_updated['date'])

In [44]:
news_sentiment_fg = fs.get_or_create_feature_group(
    name='news_sentiment_updated',
    description='News sentiment from Polygon',
    version=1,
    primary_key=['ticker'],
    event_time=['date'],
    online_enabled=False,
)

2024-05-06 11:35:39,174 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [45]:
news_sentiment_fg.insert(news_df_updated)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/549016/fs/544838/fg/785773


Uploading Dataframe: 0.00% |          | Rows 0/66 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/news_sentiment_updated_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17bc96990>, None)